In [ ]:
# !pip install google-generativeai langchain langchain-google-genai tavily-python
# !pip install faiss-cpu beautifulsoup4 pypdf
# !pip install langchain-community
# !pip -q install openai

In [ ]:
# !pip install -q langtrace-python-sdk

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

In [ ]:
from google.colab import userdata
from langtrace_python_sdk import langtrace

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
langtrace.init(api_key = userdata.get('LANGTRACE_API_KEY'))

ERROR:opentelemetry.instrumentation.instrumentor:DependencyConflict: requested: "groq >= 0.5.0" but found: "None"
ERROR:opentelemetry.instrumentation.instrumentor:DependencyConflict: requested: "pinecone-client >= 3.1.0" but found: "None"


Initializing Langtrace SDK..


ERROR:opentelemetry.instrumentation.instrumentor:DependencyConflict: requested: "llama-index >= 0.10.0" but found: "None"
ERROR:opentelemetry.instrumentation.instrumentor:DependencyConflict: requested: "chromadb >= 0.4.23" but found: "None"
ERROR:opentelemetry.instrumentation.instrumentor:DependencyConflict: requested: "qdrant-client >= 1.9.0" but found: "None"
ERROR:opentelemetry.instrumentation.instrumentor:DependencyConflict: requested: "langgraph >= 0.0.39" but found: "None"
ERROR:opentelemetry.instrumentation.instrumentor:DependencyConflict: requested: "anthropic >= 0.19.1" but found: "None"
ERROR:opentelemetry.instrumentation.instrumentor:DependencyConflict: requested: "cohere >= 5.0.0" but found: "None"
ERROR:opentelemetry.instrumentation.instrumentor:DependencyConflict: requested: "weaviate-client >= 4.6.1" but found: "None"
ERROR:opentelemetry.instrumentation.instrumentor:DependencyConflict: requested: "ollama >= 0.2.0, < 1" but found: "None"
ERROR:opentelemetry.instrumentatio

Exporting spans to Langtrace cloud..


### Load the Data

In [ ]:
loader = PyPDFLoader("Transformers.pdf")
docs = loader.load_and_split()

In [ ]:
len(docs)

12

### Split into Chunks

In [ ]:
%pip install --upgrade --quiet langchain-text-splitters tiktoken

Exported 8 spans successfully.


In [ ]:
# always split on tokens, not characters
# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/split_by_token/

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=500,
    chunk_overlap=100)

documents = text_splitter.split_documents(docs)

documents[0:5]

[Document(metadata={'source': 'Transformers.pdf', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in q

In [ ]:
print(f"Number of documents: {len(documents)}")

Number of documents: 24


In [ ]:
print(documents[5].page_content)

Figure 1: The Transformer - model architecture.
wise fully connected feed-forward network. We employ a residual connection [ 10] around each of
the two sub-layers, followed by layer normalization [ 1]. That is, the output of each sub-layer is
LayerNorm( x+ Sublayer( x)), where Sublayer(x)is the function implemented by the sub-layer
itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding
layers, produce outputs of dimension dmodel = 512 .
Decoder: The decoder is also composed of a stack of N= 6identical layers. In addition to the two
sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head
attention over the output of the encoder stack. Similar to the encoder, we employ residual connections
around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This
masking, combine

### Embeddings Model

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7ff20020c790>, model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None)

In [ ]:
embeddings.embed_query(documents[0].page_content)
print(f"lenghth of embedding: {len(embeddings.embed_query(documents[0].page_content))}")

lenghth of embedding: 768


In [ ]:
vector = FAISS.from_documents(documents, embeddings)


In [ ]:
retriever = vector.as_retriever()


In [ ]:
retriever.invoke("What is the Transformer Architecture?")


[Document(metadata={'source': 'Transformers.pdf', 'page': 1}, page_content='[9], consuming the previously generated symbols as additional input when generating the next.\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1 Encoder and Decoder Stacks\nEncoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two\nsub-layers. The ﬁrst is a multi-head self-attention mechanism, and the second is a simple, position-\n2'),
 Document(metadata={'source': 'Transformers.pdf', 'page': 1}, page_content='it more difﬁcult to learn dependencies between distant positions [ 11]. In the Transformer this is\nreduced to a constant number of operations, albeit at the cost of reduced effective resolution due\nto averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as\ndescribed 

#### Create a retrieval tool

In [ ]:
# create tool for retrieving documents
from langchain.tools.retriever import create_retriever_tool

In [ ]:
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="retriever_tool",
    description="""For any information related to transformers architecture, use this tool.
    Useful for retrieving information related to the Attention is all you need paper.""",
)


#### Create a search tool

In [ ]:
search_tool = TavilySearchResults()

In [ ]:
search_tool

TavilySearchResults()

In [ ]:
search_tool.dict()


{'name': 'tavily_search_results_json',
 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
 'args_schema': langchain_community.tools.tavily_search.tool.TavilyInput,
 'return_direct': False,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'handle_tool_error': False,
 'handle_validation_error': False,
 'response_format': 'content_and_artifact',
 'max_results': 5,
 'search_depth': 'advanced',
 'include_domains': [],
 'exclude_domains': [],
 'include_answer': False,
 'include_raw_content': False,
 'include_images': False,
 'api_wrapper': {'tavily_api_key': SecretStr('**********')}}

#### Create list of tools

In [ ]:
tools = [search_tool, retriever_tool]


#### LLM - Gemini Pro Experimental

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.environ["GOOGLE_API_KEY"])
# llm = GoogleGenerativeAI(model="gemini-1.5-pro-exp-0801", google_api_key=os.environ["GOOGLE_API_KEY"])

In [ ]:
llm

GoogleGenerativeAI(model='gemini-1.5-pro', google_api_key=SecretStr('**********'), client=genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
))

### Create The AGENT

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [ ]:
!pip -q install langchainhub

In [ ]:
prompt = hub.pull("hwchase17/react")

In [ ]:
print(prompt.template)


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)


In [ ]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\nretriever_tool(query: 'str', *, retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ff2000c1ff0>), document_prompt: 'BasePromptTemplate' = PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator: 'str' = '\\n\\n', callbacks: 'Callbacks' = None) -> 'str' - For any information related to transformers architecture, use this tool. \n    Useful for retrieving information related to the Attention is all you need paper.", 'tool_name

#### Create the Agent Executor

In [ ]:
agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [ ]:
agent_executer

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\nretriever_tool(query: 'str', *, retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ff2000c1ff0>), document_prompt: 'BasePromptTemplate' = PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator: 'str' = '\\n\\n', callbacks: 'Callbacks' = None) -> 'str' - For any information related to transformers architecture, use this tool. \n    Useful for retrieving information rel

In [ ]:
agent_executer.invoke({"input":"What is the current weather at LA?"})




> Entering new AgentExecutor chain...
Question: What is the current weather at LA?
Thought: I need to get the current weather information, so I should use the search engine.
Action: tavily_search_results_json
Action Input: current weather in Los Angeles[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Los Angeles', 'region': 'California', 'country': 'United States of America', 'lat': 34.05, 'lon': -118.24, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1723010122, 'localtime': '2024-08-06 22:55'}, 'current': {'last_updated_epoch': 1723009500, 'last_updated': '2024-08-06 22:45', 'temp_c': 24.9, 'temp_f': 76.7, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 152, 'wind_dir': 'SSE', 'pressure_mb': 1014.0, 'pressure_in': 29.96, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 56, 'cloud': 0, 'feelslike_c': 26.0, 'feelslike_f': 78.7, 'windchi

{'input': 'What is the current weather at LA?',
 'output': 'The current weather in Los Angeles is 24.9 degrees Celsius or 76.7 degrees Fahrenheit and Clear.'}

In [ ]:
agent_executer.invoke({"input":"Can you give me a list of gold medals per country in the current Olympics? "})



> Entering new AgentExecutor chain...
Question: Can you give me a list of gold medals per country in the current Olympics? 
Thought: I need to get the latest information about the Olympics to answer this question. 
Action: tavily_search_results_json
Action Input: olympic medal count 2024 [{'url': 'https://www.usatoday.com/story/sports/olympics/2024/08/05/olympic-medals-today-medal-count-paris-games-aug-5/74668390007/', 'content': "What is the medal count at the 2024 Paris Olympics? Here's the overall medal count on Monday. The U.S. leads the way with 75 total medals, and China leads with the most golds at 21."}, {'url': 'https://www.nytimes.com/interactive/2024/world/olympics-results-medals.html', 'content': 'The latest medal count and results for the Paris Olympics. ... Olympics Results. July 27, 2024. Leer en español. Share full article. Medal Count.'}, {'url': 'https://apnews.com/article/olympics-2024-medal-winners-today-b9522fd1223ae6599569ffe1ee48cc62', 'content': "PARIS (AP) — 

{'input': 'Can you give me a list of gold medals per country in the current Olympics? ',
 'output': 'According to the official medal standings for the Paris 2024 Olympics as of August 5, 2024, the USA and China are tied for the lead with 21 gold medals each.'}

In [ ]:
agent_executer.invoke({"input":"Can you explain the concept of Attention in transformers?"})



> Entering new AgentExecutor chain...
Thought: The question asks for an explanation of "Attention" in the context of transformers. This is a concept related to the "Attention is all you need" paper, so I should use the `retriever_tool`. 

Action: retriever_tool
Action Input: 'What is the concept of Attention in transformers, as described in the "Attention is all you need" paper?'it more difﬁcult to learn dependencies between distant positions [ 11]. In the Transformer this is
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summariz

{'input': 'Can you explain the concept of Attention in transformers?',
 'output': 'The attention weights in "Scaled Dot-Product Attention" are calculated using the following formula:\n\n```\nAttention(Q, K, V) = softmax(Q*K^T / sqrt(d_k)) * V\n```\n\nHere\'s a breakdown of the formula:\n\n* **Q (Queries):**  Represents the matrix of query vectors. Each query vector represents a specific word or position in the input sequence.\n* **K (Keys):** Represents the matrix of key vectors. Each key vector is associated with a specific word or position in the input sequence.\n* **V (Values):** Represents the matrix of value vectors. Each value vector contains information about a specific word or position in the input sequence.\n* **d_k:** Represents the dimension of the key vectors.\n\n**Calculation Steps:**\n\n1. **Dot Product (Q * K^T):**  The dot product of each query vector with all key vectors is calculated. This measures the similarity between each query and key.\n2. **Scaling (/ sqrt(d_k))